# Part 3: Data Analytics

## Step 1: Selecting real-world dataset

The dataset that will be used today are the stock market data of TESLA and some companies related to it. It is extracted using yfinance API. yfinance is an open source library developed by Ran Aroussi as a means to access the financial data available on Yahoo Finance[1]. Out of the many variables extracted, we will be focusing on Closing Price and Volume of shares traded. 

In [ ]:
#installing needed libraries
pip install yfinance

In [ ]:
pip install keras

In [ ]:
pip install tensorflow

In [ ]:
# Importing packages
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from keras.models import Sequential,load_model
from keras.layers import Dense, LSTM, Dropout

The stock market data is extracted by creating a ticker object for the particular stocks we need data for. The primary company I am interested is TESLA (TSLA). They are growing at a tremendous pace and is currently the biggest electric vehicle manufacturer in the world [2]. I am also interested to know the effect of TSLA share price have on its main battery supplier, Panasonic (PCRFY) [3]. I also intend to find the effect of TSLA share price on its competitor in the field of vehicle manufacturing (Ford(F) and General Motors(GM)) and energy (Royal Dutch Shell(RDS-B) and BP(BP)). We will be extracting data of last 5 years.

In [ ]:
# Setting the start and end date
start_date = '2016-11-30'
end_date = '2021-12-01'

# Set the ticker
ticker = 'TSLA','PCRFY','GM','F','RDS-B','BP'

# Get the data
data = yf.download(ticker, start_date, end_date,group_by='tickers')

# Print data
data

In [ ]:
# Saving the data into a csv file
data.to_csv('data.csv')

## Step 2: Data preparation and Cleaning

First step is we will be checking there are any null values in the dataset. We will also create 3 additional columns for the variables Percentage change of Price(%changeprice), Percentage change of volume(%changevolume) and Cumulative Return of the stock(CumulativeRet).

In [ ]:
# Loading the dataset into a data fram
stock_data = pd.read_csv('data.csv', header=[0, 1], index_col=0)
# Print dataset
stock_data

In [ ]:
# Print the shape of dataset
stock_data.shape

In [ ]:
# Checking no. of null values
stock_data.isnull().sum()

In [ ]:
# Checking any numeric data is not string
stock_data.dtypes.value_counts()

###### Pre-Processing Completed

We can see that there is no Null value in the data, and also all the stock values are not strings either. Both of these are very good indicators that all values are present.

###### Creation of additional columns

In [ ]:
# Calculating daily change of price
tesladailychanges= stock_data[( 'TSLA', 'Adj Close')]
teslapercentagedailychange= tesladailychanges.pct_change(periods=1)
stock_data['TSLA','%changeprice'] = teslapercentagedailychange

panasonicdailychanges= stock_data[( 'PCRFY', 'Adj Close')]
panasonicpercentagedailychange= panasonicdailychanges.pct_change(periods=1)
stock_data['PCRFY','%changeprice'] = panasonicpercentagedailychange

GMdailychanges= stock_data[( 'GM', 'Adj Close')]
GMpercentagedailychange= GMdailychanges.pct_change(periods=1)
stock_data['GM','%changeprice'] = GMpercentagedailychange

forddailychanges= stock_data[( 'F', 'Adj Close')]
fordpercentagedailychange= forddailychanges.pct_change(periods=1)
stock_data['F','%changeprice'] = fordpercentagedailychange

shelldailychanges= stock_data[( 'RDS-B', 'Adj Close')]
shellpercentagedailychange= shelldailychanges.pct_change(periods=1)
stock_data['RDS-B','%changeprice'] = shellpercentagedailychange

bpdailychanges= stock_data[( 'BP', 'Adj Close')]
bppercentagedailychange= bpdailychanges.pct_change(periods=1)
stock_data['BP','%changeprice'] = bppercentagedailychange

stock_data = stock_data.sort_index(axis=1)

In [ ]:
# Calculating daily change of volume traded
teslavolumechanges= stock_data[( 'TSLA', 'Volume')]
teslapercentagevolumechange= teslavolumechanges.pct_change(periods=1)
stock_data['TSLA','%changevolume'] = teslapercentagevolumechange

panasonicvolumechanges= stock_data[( 'PCRFY', 'Volume')]
panasonicpercentagevolumechange= panasonicvolumechanges.pct_change(periods=1)
stock_data['PCRFY','%changevolume'] = panasonicpercentagevolumechange

GMvolumechanges= stock_data[( 'GM', 'Volume')]
GMpercentagevolumechange= GMvolumechanges.pct_change(periods=1)
stock_data['GM','%changevolume'] = GMpercentagevolumechange

fordvolumechanges= stock_data[( 'F', 'Volume')]
fordpercentagevolumechange= fordvolumechanges.pct_change(periods=1)
stock_data['F','%changevolume'] = fordpercentagevolumechange

shellvolumechanges= stock_data[( 'RDS-B', 'Volume')]
shellpercentagevolumechange= shellvolumechanges.pct_change(periods=1)
stock_data['RDS-B','%changevolume'] = shellpercentagevolumechange

bpvolumechanges= stock_data[( 'BP', 'Volume')]
bppercentagevolumechange= bpvolumechanges.pct_change(periods=1)
stock_data['BP','%changevolume'] = bppercentagevolumechange

stock_data = stock_data.sort_index(axis=1)

In [ ]:
# Calculating Cumulative Return of stock
stock_data['TSLA','CumulativeRet'] = (1+ stock_data[( 'TSLA', '%changeprice')]).cumprod()

stock_data['PCRFY','CumulativeRet'] = (1+ stock_data[( 'PCRFY', '%changeprice')]).cumprod()

stock_data['GM','CumulativeRet'] = (1+ stock_data[( 'GM', '%changeprice')]).cumprod()

stock_data['F','CumulativeRet'] = (1+ stock_data[( 'F', '%changeprice')]).cumprod()

stock_data['RDS-B','CumulativeRet'] = (1+ stock_data[( 'RDS-B', '%changeprice')]).cumprod()

stock_data['BP','CumulativeRet'] = (1+ stock_data[( 'BP', '%changeprice')]).cumprod()

stock_data = stock_data.sort_index(axis=1)

In [ ]:
# Printing stock data
stock_data

## Step 3: Exploratory Analysis

Let us look at the Adjusted Close price of Tesla and rest of the companies for the last 5 years.

In [ ]:
# Plot the adjusted close price
stock_data[('TSLA','Adj Close')].plot(figsize=(16, 10))

# Define the label for the title of the figure
plt.title("Adjusted Close Price of Tesla", fontsize=16)

# Define the labels for x-axis and y-axis
plt.ylabel('Price', fontsize=14)
plt.xlabel('Year', fontsize=14)

# Plot the grid lines
plt.grid(which="major", color='k', linestyle='-.', linewidth=0.5)

# Show the plot
plt.show()

In [ ]:
# Getting adjusted close price
AdjClose=data.iloc[:, stock_data.columns.get_level_values(1)=='Adj Close']

AdjClose.plot(figsize=(16, 10))

# Show the legend
plt.legend()

# Define the label for the title of the figure
plt.title("Adjusted Close Price", fontsize=16)

# Define the labels for x-axis and y-axis
plt.ylabel('Price', fontsize=14)
plt.xlabel('Year', fontsize=14)

# Plot the grid lines
plt.grid(which="major", color='k', linestyle='-.', linewidth=0.5)
plt.show()

From the above graph it can be seen that the Adjusted Close price of Tesla was approximately in the same range as rest of the companies that we are comparing until early 2020. Then it started to rise significantly whereas share price of the rest of the companies remained more or less same. 

###### Question 1: Does change in stock prices affect the volume of stock traded of Tesla? What is the relation between volume and close price of Tesla? 

I intend to find that does the change in price of Tesla share have any effect on the volume of shares traded. It is generally expected that more trading of shares will occur when there is a significant change in price as people will rush to take advantage of the price swing.

In [ ]:
# Plot the adjusted close price
stock_data[('TSLA','%changevolume')].plot(figsize=(16, 10))

# Define the label for the title of the figure
plt.title("Daily % change of Volume of Tesla shares traded", fontsize=16)

# Define the labels for x-axis and y-axis
plt.ylabel('% change of Volume', fontsize=14)
plt.xlabel('Year', fontsize=14)
plt.ticklabel_format(useOffset=False, style='plain',axis='y')
# Plot the grid lines
plt.grid(which="major", color='k', linestyle='-.', linewidth=0.5)

# Show the plot
plt.show()


stock_data[('TSLA','%changeprice')].plot(figsize=(17,10))
plt.suptitle('Daily % change of Tesla. stock price', fontsize=16, color='black')
plt.ylabel('% change of share price', fontsize=14)
plt.xlabel('Year', fontsize=14)
plt.grid(which="major", color='k', linestyle='-.', linewidth=0.5)

plt.show() 

The y-axis of both graphs represents the percentage change in the value. This is needed because the daily volume of shares traded will be in the range of hundreds of thousands, and the daily price movement is normally a few dollars. We may compare the two datasets on a similar scale by converting to percentage change. There is no visible association between the number of shares exchanged and the daily price change, as seen in the above graph.

In [ ]:
# Scatter plot between the values
plt.figure(figsize=(16,10))
plt.scatter(stock_data[('TSLA','%changevolume')],stock_data[('TSLA','%changeprice')])
plt.suptitle('Scatter plot between %change in price and volume traded of Tesla stock', fontsize=16, color='black')
plt.ylabel('% change in price of stock', fontsize=14)
plt.xlabel('% change in Volume Traded', fontsize=14)
plt.grid(which="major", color='k', linestyle='-.', linewidth=0.5)
plt.ticklabel_format(useOffset=False, style='plain')

Just from the above scatter plot, it doesn't look like there is much relationship between share price and volume traded of Tesla stock. 

In [ ]:
# Linear regression model
a= stock_data[('TSLA','%changevolume')]
b= stock_data[('TSLA','%changeprice')]
x=np.array(a[1:1258]).reshape((-1,1))
y=b[1:1258]
model = LinearRegression()
model.fit(x, y)
r_sq = model.score(x, y)
print('coefficient of determination:', r_sq)

The very low value of coefficient of determination tells that there is no relation between the variables. With all the tests done we have reached the same conclusion that there is no relation between change in share price and volume traded of Tesla, which is quite surprising as we would expect that there would be more trading activity when there is a drastic change in price as people will try to benefit from the said change in price.

###### Question 2: Does change in price of Tesla stock has any effect on its main battery supplier and competitors? What is the volatility and correlation between these stocks? 

Market performance of one company can have some influence on the rest of the market. I intend to find the effect of Tesla share price have on its main battery supplier, Panasonic i.e., will increase in Tesla share price can have positive impact on share price of Panasonic. Also, I will look into the effect of Tesla share price on its competitors. The competitors that we will be looking in the vehicle manufacturing industry are Ford and GM. They are the biggest gasoline vehicle producer in the home country of Tesla. I also plan to find any relation between share price of petroleum companies (Royal Dutch Shell and BP) and Tesla. Since Tesla is an electric vehicle manufacture, better performance of the company may have negative impact on the share price of petroleum companies. I aim to find that this is the case.

In [ ]:
# Getting the percentage change of price
percentagechange=data.iloc[:, stock_data.columns.get_level_values(1)=='%changeprice']

percentagechange.plot(subplots = True,figsize=(16, 16))

# Show the legend
plt.legend()

# Define the label for the title of the figure
plt.suptitle("Daily Percentage change of stock price of all the companies", fontsize=16)
# Define the labels for x-axis and y-axis
plt.ylabel('% change in Price', fontsize=14)
plt.xlabel('Year', fontsize=14)

# Plot the grid lines
plt.grid(which="major", color='k', linestyle='-.', linewidth=0.5)
plt.show()

From the above graph it can be seen that percentage change of share price of stock is not that much related to each other except for early 2020. There is huge spike in graph for each company. I think this can be attributed to the COVID-19 lockdown and its effect on the markets. 

In [ ]:
#Plotting scatter matrix
from pandas.plotting import scatter_matrix
comparison=pd.concat([stock_data['TSLA','Open'],stock_data['PCRFY','Open'],stock_data['GM','Open'],stock_data['F','Open'],stock_data['RDS-B','Open'],stock_data['BP','Open']],axis=1)
comparison.columns=['Tesla','Panasonic','GM','Ford','Shell','BP']
scatter_matrix(comparison,figsize=(16,16),hist_kwds={'bins':50})

From the above scatter matrix, it is clear that share price of Tesla have almost strong non-linear relationship with rest of the companies. The only strong linear relationship that can be found is between the share price of Ford and GM and also with Shell and BP.

In [ ]:
# Checking the volatility of the stock
stock_data['TSLA','%changeprice'].hist(bins=50,label='Tesla',alpha=0.5,figsize=(16,16))
stock_data['PCRFY','%changeprice'].hist(bins=50,label='Panasonic',alpha=0.5)
stock_data['GM','%changeprice'].hist(bins=50,label='GM',alpha=0.5)
stock_data['F','%changeprice'].hist(bins=50,label='Ford',alpha=0.5)
stock_data['RDS-B','%changeprice'].hist(bins=50,label='Shell',alpha=0.5)
stock_data['BP','%changeprice'].hist(bins=50,label='BP',alpha=0.5)
plt.legend()

The curve corresponding to Tesla is more wider than rest. Hence share price of Tesla is more volatile than rest of the companies.

In [ ]:
# getting kde curve
stock_data['TSLA','%changeprice'].plot(kind='kde',label='Tesla',alpha=0.5,figsize=(16,12))
stock_data['PCRFY','%changeprice'].plot(kind='kde',label='Panasonic',alpha=0.5)
stock_data['GM','%changeprice'].plot(kind='kde',label='GM',alpha=0.5)
stock_data['F','%changeprice'].plot(kind='kde',label='Ford',alpha=0.5)
stock_data['RDS-B','%changeprice'].plot(kind='kde',label='Shell',alpha=0.5)
stock_data['BP','%changeprice'].plot(kind='kde',label='BP',alpha=0.5)
plt.grid(which="major", color='k', linestyle='-.', linewidth=0.5)
plt.legend()

From the above graph, it can be seen that Tesla has wider curve. Hence, we can definitely confirm that Tesla stock is most volatile. 

In [ ]:
# Getting boxplots
returncomparison=pd.concat([stock_data['TSLA','%changeprice'],stock_data['PCRFY','%changeprice'],stock_data['GM','%changeprice'],stock_data['F','%changeprice'],stock_data['RDS-B','%changeprice'],stock_data['BP','%changeprice']],axis=1)
returncomparison.columns=['Tesla','Panasonic','GM','Ford','Shell','BP']
returncomparison.plot(kind='box',figsize=(16,16))

From above box plot, it is clear that Percentage change in share price of Tesla is more spread out than rest of the companies.

In [ ]:
# Finding the correlation between the daily returns of the company
scatter_matrix(returncomparison,figsize=(16,16),hist_kwds={'bins':50},alpha=0.25)

From the scatter matrix, it can be seen that there is no correlation between the daily returns of Tesla and rest of the companies. Just like before the only relation that can be deduced is the positive relation between the daily returns of Ford and GM and also Shell and BP.

###### Question 3: What is the cumulative return of each company? Is it possible to predict future stock price of Tesla even though it is highly volatile?

I intend to find the cumulative return for the period of 5 years i.e., which company provides highest return. I also aim to find that whether it is possible to build a prediction model using past data of Tesla. From the above graphs it is clear that Tesla stock is highly volatile. So, my objective is to find that is it possible to build sufficiently good prediction model using machine learning with the past volatile data.

In [ ]:
# Getting the cumulative return
stock_data['TSLA','CumulativeRet'].plot(label='Tesla',figsize=(16,10))
stock_data['GM','CumulativeRet'].plot(label='GM',figsize=(16,10))
stock_data['F','CumulativeRet'].plot(label='Ford',figsize=(16,10))
stock_data['PCRFY','CumulativeRet'].plot(label='Panasonic',figsize=(16,10))
stock_data['RDS-B','CumulativeRet'].plot(label='Shell',figsize=(16,10))
stock_data['BP','CumulativeRet'].plot(label='BP',figsize=(16,10))
plt.title("Cumulative Return vs Time")
plt.grid(which="major", color='k', linestyle='-.', linewidth=0.5)
plt.legend()

From the above graph, it is clear that Tesla provides with the highest return. 1 pound invested at the end 2016 would have grown to around 30 pounds at the end of 2021. Whereas the rest of the companies under consideration would just have increased maximum by 2.5x.

###### Building prediction model with LSTM[4]

In [ ]:
# Dropping Null values
data=stock_data.dropna()

In [ ]:
#Create a new dataframe with only the TSLA data

tesla_data = stock_data['TSLA']

#Create a new dataframe with only the 'Close' column
close_data = tesla_data.filter(['Close'])

In [ ]:
#Convert the dataframe to a numpy
dataset = close_data.values
#Get the number of rows to train model
training_data_len = math.ceil(len(dataset)* 0.8)
training_data_len

In [ ]:
#Scaling the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

#Create the training data set
#Create the scaled training data set
train_data = scaled_data[0:training_data_len,:]
#Split the data into x_train and y_train data sets
x_train = []
y_train = []

for i in range(60,len(train_data)):
    x_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])

In [ ]:
#Convert the x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train),np.array(y_train)
x_train.shape

#Reshape the data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

In [ ]:
# Building LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(x_train.shape[1], 1)))

model.add(LSTM(units=64,return_sequences=False))


model.add(Dense(25))
model.add(Dense(1))

In [ ]:
#Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')
#Train the model
model.fit(x_train, y_train, epochs=10, batch_size=10)

In [ ]:
#Create testing dataset
#Create a new array containing scaled values from index 947 to 1259
test_data = scaled_data[training_data_len-60:,:]
#Create the data sets x_test and y_test
x_test =[]
y_test = dataset[training_data_len:,:]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i,0])

In [ ]:
#Convert the data to a numpy
x_test = np.array(x_test)

#Reshape the data
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
#Model's predicted price values
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
#Get the Root Mean Squared Error(RMSE)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse

In [ ]:
#Plot the data
train = close_data[:training_data_len]
valid = close_data[training_data_len:]
valid['Predictions'] = predictions

In [ ]:
#Visualize the data
plt.figure(figsize=(16,10))
plt.title('Model')
plt.xlabel('Date',fontsize=14)
plt.ylabel('Close Price',fontsize=14)
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])
plt.legend(['Train','Val','Predictions'], loc='lower right')
plt.show()

From above graph, it can be seen that stock price can sometimes be calculated with good accuracy.

In [ ]:
#show the valid and predicted prices
valid

###### Predicting next day price

In [ ]:
#Get the last 60 days closing price values 
last_60_days = close_data[-60:].values
#Scale the data to values between 0 and 1
last_60_days_scaled = scaler.transform(last_60_days)
#Create empty list
X_test = []
#Append the list
X_test.append(last_60_days_scaled)
#Convert the X_test set to numpy array
X_test = np.array(X_test)
#Reshape the data
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
#Get the predicted scaled price
predicted_price = model.predict(X_test)
predicted_price = scaler.inverse_transform(predicted_price)
print(predicted_price)

In [ ]:
upper_band= predicted_price + rmse
lower_band= predicted_price - rmse

actual_value = yf.download('TSLA', start_date='2021-11-30', end_date='2021-12-10')
desired_value =actual_value.loc[pd.DatetimeIndex(['2021-12-01']), 'Close'].values[0]
desired_value 

In [ ]:
if desired_value>lower_band and desired_value<upper_band:
    print("The predicted value is within the standard deviation of errors.")
else:
    print("The predicted value is outside the standard deviation of errors. You may need to retrain the model.")

It is observed that stock prices can be predicted with very good accuracy. But sometimes the trained model generates large RMSE value and results in poor accuracy. Retraining the model again can solve this issue.